In [9]:
import os
import pandas as pd
from sqlalchemy import create_engine

HOST = os.getenv("POSTGRES_HOST", default="localhost")
PORT = os.getenv("POSTGRES_PORT", default=5429)
DATABASE = os.getenv("POSTGRES_DB", default="osm-paper")
USER = os.getenv("POSTGRES_USER", default="osm-paper")
PASSWORD = os.getenv("POSTGRES_PASSWORD", default="osm-paper")

## External Reference Datasets

In [11]:
con = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
query = """
    with unnested_data_urban_centers as (
            select 
                urban_center_id
                ,region_wb 
                ,unnest(external_reference_sources) as external_reference_source
            from full_urban_centers
        ),
        unnested_data_urban_centers_grid as (
            select
                urban_center_id
                ,region_wb
                ,unnest(reference_source) as external_reference_source
                ,count(*) as n_grids
            from full_urban_centers_grid fucg 
            group by urban_center_id, region_wb, external_reference_source
        )
        select 
            a.external_reference_source
            ,a.region_wb 
            ,count(*) as n_urban_centers
            ,sum(b.n_grids) as n_grid_cells
        from unnested_data_urban_centers a
        left join unnested_data_urban_centers_grid b on
            a.urban_center_id = b.urban_center_id and
            a.region_wb = b.region_wb and
            a.external_reference_source = b.external_reference_source
        where a.external_reference_source is not null
        group by a.region_wb, a.external_reference_source
        order by a.region_wb 
"""
df = pd.read_sql_query(query, con=con)
display(df)

,external_reference_source,region_wb,n_urban_centers,n_grid_cells
0,lds,East Asia & Pacific,8,941.0
1,gsi,East Asia & Pacific,108,22342.0
2,nsdi,East Asia & Pacific,37,5258.0
3,gugik,Europe & Central Asia,48,3221.0
4,cuzk,Europe & Central Asia,12,717.0
5,estonia,Europe & Central Asia,2,130.0
6,os,Europe & Central Asia,135,10845.0
7,hausumringe,Europe & Central Asia,24,4203.0
8,ign,Europe & Central Asia,71,6292.0
9,nl,Europe & Central Asia,38,2687.0


## Microsoft Buildings

In [15]:
con = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
query = """
    select
        a.region_wb 
        ,count(distinct a.urban_center_id) as n_urban_centers
        ,count(distinct b.grid_fid) as n_grid_cells
        ,array_agg(distinct a.iso_a3) as iso_a3_list
    from full_urban_centers a
    left join full_urban_centers_grid b on
        a.urban_center_id = b.urban_center_id 
    where a.microsoft_building_area_sqkm is not null and b.urban_center_id is not null
    group by a.region_wb 
"""
df = pd.read_sql_query(query, con=con)
display(df)
display(df.to_string())

,region_wb,n_urban_centers,n_grid_cells,iso_a3_list
0,East Asia & Pacific,825,58147,"[AUS, IDN, KHM, LAO, MMR, MNG, MYS, PHL, PRK, ..."
1,Europe & Central Asia,1042,69954,"[-99, ALB, ARM, AUT, AZE, BEL, BGR, BIH, BLR, ..."
2,Latin America & Caribbean,877,52135,"[ARG, BHS, BLZ, BOL, BRA, BRB, CHL, COL, CRI, ..."
3,Middle East & North Africa,666,31948,"[ARE, DJI, DZA, EGY, IRN, IRQ, ISR, JOR, LBN, ..."
4,North America,367,100210,"[CAN, USA]"
5,South Asia,1677,56421,"[AFG, BGD, BTN, IND, LKA, MDV, NPL, PAK]"
6,Sub-Saharan Africa,1527,43376,"[AGO, BDI, BEN, BFA, BWA, CAF, CIV, CMR, COD, ..."


'                    region_wb  n_urban_centers  n_grid_cells                                                                                                                                                                                                                                            iso_a3_list\n0         East Asia & Pacific              825         58147                                                                                                                                                                                                [AUS, IDN, KHM, LAO, MMR, MNG, MYS, PHL, PRK, THA, VNM]\n1       Europe & Central Asia             1042         69954  [-99, ALB, ARM, AUT, AZE, BEL, BGR, BIH, BLR, CHE, CYP, CZE, DEU, DNK, ESP, EST, FIN, FRA, GBR, GEO, GRC, HRV, HUN, IRL, ISL, ITA, KAZ, KGZ, LTU, LUX, LVA, MDA, MKD, MNE, NLD, NOR, POL, PRT, ROU, RUS, SRB, SVK, SVN, SWE, TJK, TKM, TUR, UKR, UZB]\n2   Latin America & Caribbean              877         52135            